# Installs

In [52]:
%pip install datasets

In [53]:
%pip install rouge_score

In [54]:
%pip install sacrebleu

In [55]:
%pip install -U nltk

In [56]:
%pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-qqira52_
  Running command git clone -q https://github.com/google-research/bleurt.git /tmp/pip-req-build-qqira52_


In [57]:
%pip install bert_score

In [58]:
!git clone "https://github.com/neural-dialogue-metrics/Distinct-N.git"

fatal: destination path 'Distinct-N' already exists and is not an empty directory.


In [59]:
%pip install ./Distinct-N/

Processing ./Distinct-N
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for Distinct-N: filename=Distinct_N-0.4.0-py3-none-any.whl size=8788 sha256=18161e6e22cfa929d30280c48dead795800f2a74d910d9fec50e864d05dd07c6
  Stored in directory: /root/.cache/pip/wheels/88/b9/55/7c64e9c4d989d19831389177836147b2955af02bc30a861d87
Successfully built Distinct-N
  Attempting uninstall: Distinct-N
    Found existing installation: Distinct-N 0.4.0
    Uninstalling Distinct-N-0.4.0:
      Successfully uninstalled Distinct-N-0.4.0


In [60]:
%pip install lexical-diversity

In [61]:
!git clone https://github.com/wl-research/nubia.git

fatal: destination path 'nubia' already exists and is not an empty directory.


# Load dataset

In [62]:
# Load dataset
from datasets import load_dataset
data = load_dataset("gem", "common_gen")

Reusing dataset gem (/root/.cache/huggingface/datasets/gem/common_gen/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4)


## Explore dataset structure

In [14]:
data

DatasetDict({
    train: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 67389
    })
    validation: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 993
    })
    test: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 1497
    })
    challenge_train_sample: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
    challenge_validation_sample: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
    challenge_test_scramble: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
})

In [ ]:
test_instance = data['test'][0]
test_instance

In [ ]:
validation_instance = data['validation'][133]
validation_instance

In [ ]:
challenge_test_instance = data['challenge_test_scramble'][0]
challenge_test_instance

In [ ]:
challenge_validation_instance = data['challenge_validation_sample'][0]
challenge_validation_instance

In [ ]:
concept = challenge_validation_instance.get("target")
concept

In [18]:
validation_targets = []
for i in range(501): 
  instance = data['validation'][i]
  target = instance.get("target")
  validation_targets.append(target)
validation_targets[500]

'The woman rides a bicycle down the street instead of walking.'

## Extract validation split

In [41]:
validation_data = []
validation_data = data['validation']
print(validation_data['references'][0])
print(validation_data['target'][0])

['The player stood in the field looking at the batter.', 'The coach stands along the field, looking at the goalkeeper.', 'I stood and looked across the field, peacefully.', 'Someone stands, looking around the empty field.']
The player stood in the field looking at the batter.


## Clean generated text

In [63]:
# Clean validation greedy sampling generation
data_file = "sample_data/greedy_validation_set.txt"
greedy_generated_data = []

with open(data_file, 'r') as f:
  for i, line in enumerate(f):
    line = line.strip()
    example = line.split()
    new_example = []
    for word in example:
      if word.startswith("[unused") or word.startswith("[UNK]") or word.startswith(".") or word.startswith("\\") or word.startswith("##"):
        del word
      else:
        new_example.append(word)

    complete_sentence = " ".join(new_example) + "."
    greedy_generated_data.append(complete_sentence)

greedy_generated_data[0]

'the two beautiful white black sheep that are standing out and grazing close up together together in a blue green grassy field standing together.'

In [64]:
# Clean validation top-k generation
data_file = "sample_data/sampling_validation_set.txt"
sampling_generated_data = []

with open(data_file, 'r') as f:
  for i, line in enumerate(f):
    line = line.strip()
    example = line.split()
    new_example = []
    for word in example:
      if word.startswith("[unused") or word.startswith("[UNK]") or word.startswith(".") or word.startswith("\\") or word.startswith("##"):
        del word
      else:
        new_example.append(word)

    complete_sentence = " ".join(new_example) + "."
    sampling_generated_data.append(complete_sentence)

sampling_generated_data[0]

'a herd of white sheep there are standing together in a bright green golden field , with two elephants moving together so incredibly happy together that they are all this things and look over at each summer all other together outside the airport gives postdoctoral.'

# Compute metrics


## Lexical (ROUGE 1/2/L, BLEU, Meteor), 

In [17]:
from datasets import list_metrics, load_metric
import numpy as np

metrics_list = list_metrics()
print(metrics_list)

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'comet', 'coval', 'cuad', 'f1', 'gleu', 'glue', 'indic_glue', 'matthews_correlation', 'meteor', 'pearsonr', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'wer', 'xnli']


In [23]:
rouge_scorer = load_metric("rouge")
sacrebleu_scorer = load_metric("sacrebleu")
meteor_scorer = load_metric("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [24]:
# ROUGE target
greedy_rouge_results = rouge_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target'],
    rouge_types=["rouge1", "rouge2", "rougeL"],
    use_agregator=True, use_stemmer=False,
)

print("Greedy rouge1", round(greedy_rouge_results['rouge1'].mid.fmeasure, 4))
print("Greedy rouge2", round(greedy_rouge_results['rouge2'].mid.fmeasure, 4))
print("Greedy rougeL", round(greedy_rouge_results['rougeL'].mid.fmeasure, 4))


sampling_rouge_results = rouge_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target'],
    rouge_types=["rouge1", "rouge2", "rougeL"],
    use_agregator=True, use_stemmer=False,
)

print("Sampling rouge1", round(sampling_rouge_results['rouge1'].mid.fmeasure, 4))
print("Sampling rouge2", round(sampling_rouge_results['rouge2'].mid.fmeasure, 4))
print("Sampling rougeL", round(sampling_rouge_results['rougeL'].mid.fmeasure, 4))

Greedy rouge1 0.1377
Greedy rouge2 0.008
Greedy rougeL 0.1094
Sampling rouge1 0.1428
Sampling rouge2 0.0085
Sampling rougeL 0.1069


In [25]:
greedy_rouge_results

{'rouge1': AggregateScore(low=Score(precision=0.09930407590354992, recall=0.26927737006050306, fmeasure=0.13243650810796195), mid=Score(precision=0.10404902175431105, recall=0.28078453481300086, fmeasure=0.13769484982852698), high=Score(precision=0.10893608689550802, recall=0.29231587425587124, fmeasure=0.14336645750434507)),
 'rouge2': AggregateScore(low=Score(precision=0.004899776947490292, recall=0.01435445599429267, fmeasure=0.006620560541808428), mid=Score(precision=0.006038040733274561, recall=0.017098329973993485, fmeasure=0.008005904914145066), high=Score(precision=0.007274251012223819, recall=0.019850000409813547, fmeasure=0.009520644207426794)),
 'rougeL': AggregateScore(low=Score(precision=0.08038642949765044, recall=0.20846627284812547, fmeasure=0.10505534155263364), mid=Score(precision=0.08413962791141139, recall=0.21734432006484025, fmeasure=0.10937809093982245), high=Score(precision=0.0879784047379405, recall=0.22695993731775294, fmeasure=0.1138925726296857))}

In [26]:
# BLEU target

greedy_bleu_results = []
for i, sentence in enumerate(zip(greedy_generated_data, validation_data)):
  greedy_bleu_result = sacrebleu_scorer.compute(
      predictions=[greedy_generated_data[i]],
      references=[[validation_data['target'][i]]]
  )
  greedy_bleu_results.append(greedy_bleu_result["score"])

sampling_bleu_results = []
for i, sentence in enumerate(zip(sampling_generated_data, validation_data)):
  sampling_bleu_result = sacrebleu_scorer.compute(
      predictions=[sampling_generated_data[i]],
      references=[[validation_data['target'][i]]]
  )
  sampling_bleu_results.append(sampling_bleu_result["score"])

greedy_bleu_mean = np.mean(greedy_bleu_results)
sampling_bleu_mean = np.mean(sampling_bleu_results)

print("Greedy BLEU mean", greedy_bleu_mean)
print("Sampling BLEU mean", sampling_bleu_mean)


Greedy BLEU mean 2.1781781058979433
Sampling BLEU mean 1.8177491934125327


In [27]:
# BLEU references
greedy_bleu_results = []
for i, sentence in enumerate(zip(greedy_generated_data, validation_data)):
  greedy_bleu_result = sacrebleu_scorer.compute(
      predictions=[greedy_generated_data[i]],
      references=[validation_data['references'][i]]
  )
  greedy_bleu_results.append(greedy_bleu_result["score"])

sampling_bleu_results = []
for i, sentence in enumerate(zip(sampling_generated_data, validation_data)):
  sampling_bleu_result = sacrebleu_scorer.compute(
      predictions=[sampling_generated_data[i]],
      references=[validation_data['references'][i]]
  )
  sampling_bleu_results.append(sampling_bleu_result["score"])

greedy_bleu_mean = np.mean(greedy_bleu_results)
sampling_bleu_mean = np.mean(sampling_bleu_results)

print("Greedy BLEU mean", greedy_bleu_mean)
print("Sampling BLEU mean", sampling_bleu_mean)


Greedy BLEU mean 2.880467903421246
Sampling BLEU mean 2.4564428937335037


In [28]:
### NIST score
from nltk.translate import nist_score

reference = 'un test simple'.split()
reference1 = 'un test super simple'.split()
candidate = 'un test pas trop simple'.split()

print('Sentence 1-gram: %f' % nist_score.sentence_nist([reference, reference1], candidate, 1))
print('Sentence 2-gram: %f' % nist_score.sentence_nist([reference], candidate, 2))
print('Sentence 3-gram: %f' % nist_score.sentence_nist([reference], candidate, 3))
print('Sentence 4-gram: %f' % nist_score.sentence_nist([reference], candidate, 4))
print('Sentence 5-gram: %f' % nist_score.sentence_nist([reference], candidate, 5))

print('Corpus 1-gram: %f' % nist_score.corpus_nist([[reference, reference1]], [candidate], 1))
print('Corpus 2-gram: %f' % nist_score.corpus_nist([[reference, reference1]], [candidate], 2))
print('Corpus 3-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 3))
print('Corpus 4-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 4))
print('Corpus 5-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 5))

Sentence 1-gram: 1.084413
Sentence 2-gram: 0.950978
Sentence 3-gram: 0.950978
Sentence 4-gram: 0.950978
Sentence 5-gram: 0.950978
Corpus 1-gram: 1.084413
Corpus 2-gram: 1.084413
Corpus 3-gram: 0.950978
Corpus 4-gram: 0.950978
Corpus 5-gram: 0.950978


In [29]:
# NIST
from nltk.translate import nist_score

greedy_nist_results = []
for i, sentence in enumerate(zip(validation_data, greedy_generated_data)):
  reference = validation_data['target'][i].split()
  candidate = greedy_generated_data[i].split()
  
  if len(candidate) > 4:
    greedy_nist_result = nist_score.sentence_nist(reference, candidate)
    greedy_nist_results.append(greedy_nist_result)

sampling_nist_results = []
for i, sentence in enumerate(zip(validation_data, sampling_generated_data)):
  reference = validation_data['target'][i].split()
  candidate = sampling_generated_data[i].split()
  
  if len(candidate) > 4:
    sampling_nist_result = nist_score.sentence_nist(reference, candidate)
    sampling_nist_results.append(sampling_nist_result)

    
greedy_nist_mean = np.mean(greedy_nist_results)
sampling_nist_mean = np.mean(sampling_nist_results)

print("Greedy NIST mean", greedy_nist_mean)
print("Sampling NIST mean", sampling_nist_mean)

Greedy NIST mean 0.11406768741893292
Sampling NIST mean 0.09360422933071363


In [30]:
# METEOR
greedy_meteor_results = meteor_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target']
)

sampling_meteor_results = meteor_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target']
)

print("Greedy meteor", round(greedy_meteor_results['meteor'], 4))
print("Sampling meteor", round(sampling_meteor_results['meteor'], 4))

Greedy meteor 0.123
Sampling meteor 0.1415


 ## Semantic (BERTscore, BLEURT, NUBIA)

In [18]:
import numpy as np

bleurt_scorer = load_metric('bleurt', 'bleurt-large-512')
bertscore_scorer = load_metric("bertscore")


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/bleurt-large-512/downloads/extracted/299e33e80b83c78cc60e485384c7804f6ec1fb36c2013c5078257c17a82719ca/bleurt-large-512.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


In [19]:
# BLEURT

greedy_bleurt_results = bleurt_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target']
)
sampling_bleurt_results = bleurt_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target']
)

In [20]:
# When sequence length is 512
greedy_bleurt_results

{'scores': [-1.2710376977920532,
  -1.1292105913162231,
  -1.2065376043319702,
  -1.22152841091156,
  -1.1583713293075562,
  -1.418140172958374,
  -1.2273058891296387,
  -1.2115908861160278,
  -1.0698431730270386,
  -1.3259915113449097,
  -1.2479339838027954,
  -1.2693127393722534,
  -1.2190600633621216,
  -1.3143789768218994,
  -0.7860374450683594,
  -1.1690196990966797,
  -0.3411678373813629,
  -1.4099310636520386,
  -1.09270441532135,
  -1.1134439706802368,
  -1.3556106090545654,
  -1.1926504373550415,
  -1.4764915704727173,
  -1.2267266511917114,
  -1.3520257472991943,
  -1.2992271184921265,
  -1.2169575691223145,
  -1.196334958076477,
  -0.8679672479629517,
  -1.478190302848816,
  -1.2425706386566162,
  -1.2912040948867798,
  -1.0058529376983643,
  -1.3013774156570435,
  -1.3733528852462769,
  -1.4480031728744507,
  -1.143324375152588,
  -1.4302244186401367,
  -1.4287177324295044,
  -1.2296942472457886,
  -1.2832646369934082,
  -1.1825398206710815,
  -1.1469430923461914,
  -0.9187

In [33]:
# When sequece length is 128
greedy_bleurt_results

{'scores': [-1.4496817588806152,
  -1.3335041999816895,
  -1.5066707134246826,
  -1.3614387512207031,
  -1.4573159217834473,
  -1.5093721151351929,
  -1.298492670059204,
  -1.3099042177200317,
  -0.9760408997535706,
  -1.3199212551116943,
  -1.4218918085098267,
  -1.5418888330459595,
  -1.3614728450775146,
  -1.4002652168273926,
  -0.6932849287986755,
  -1.5293967723846436,
  -0.8330033421516418,
  -1.5051426887512207,
  -1.4627039432525635,
  -1.280038833618164,
  -1.5779520273208618,
  -1.5528712272644043,
  -1.681053638458252,
  -1.536818504333496,
  -1.484250783920288,
  -1.335343837738037,
  -1.3663277626037598,
  -1.2549726963043213,
  -0.8573294281959534,
  -1.5844378471374512,
  -1.535088062286377,
  -1.3870999813079834,
  -1.4149284362792969,
  -1.4263299703598022,
  -1.6673035621643066,
  -1.3870353698730469,
  -1.434201717376709,
  -1.7423479557037354,
  -1.689113736152649,
  -1.5531506538391113,
  -1.6985557079315186,
  -1.395650863647461,
  -1.4805607795715332,
  -1.219984

In [22]:
# When seq len = 512
sampling_bleurt_results

{'scores': [-1.397591471672058,
  -1.1856704950332642,
  -1.3936225175857544,
  -1.2488330602645874,
  -1.3540868759155273,
  -1.3740642070770264,
  -1.1464332342147827,
  -1.1972113847732544,
  -1.394267201423645,
  -1.165199875831604,
  -1.0895088911056519,
  -1.3000856637954712,
  -1.0675193071365356,
  -1.314782977104187,
  -1.4055087566375732,
  -1.3294280767440796,
  -1.2997514009475708,
  -1.4087748527526855,
  -1.3666247129440308,
  -1.1717551946640015,
  -1.3745064735412598,
  -1.1289008855819702,
  -1.2637786865234375,
  -1.0874310731887817,
  -1.3990154266357422,
  -1.2547329664230347,
  -1.374333143234253,
  -1.1655734777450562,
  -1.1071234941482544,
  -1.210280418395996,
  -1.3315833806991577,
  -1.3274582624435425,
  -1.2331676483154297,
  -1.4675761461257935,
  -1.3421577215194702,
  -1.4613555669784546,
  -1.3275560140609741,
  -1.541088342666626,
  -1.3851442337036133,
  -0.9360547065734863,
  -1.0106453895568848,
  -1.3221021890640259,
  -1.1496645212173462,
  -1.406

In [34]:
# When seq len = 128
sampling_bleurt_results

{'scores': [-1.616408348083496,
  -1.5641789436340332,
  -1.5907269716262817,
  -1.3733481168746948,
  -1.331987738609314,
  -1.4432528018951416,
  -1.2366676330566406,
  -1.3705310821533203,
  -1.4877420663833618,
  -1.3352817296981812,
  -1.2580758333206177,
  -1.833547592163086,
  -1.3031752109527588,
  -1.360670804977417,
  -1.6041107177734375,
  -1.5950250625610352,
  -1.3864672183990479,
  -1.4196665287017822,
  -1.5296692848205566,
  -1.259598731994629,
  -1.5543252229690552,
  -1.51846182346344,
  -1.6034002304077148,
  -1.3999215364456177,
  -1.3554011583328247,
  -1.47396981716156,
  -1.4617276191711426,
  -1.2019305229187012,
  -1.17335844039917,
  -1.5807466506958008,
  -1.2643544673919678,
  -1.2288320064544678,
  -1.5107676982879639,
  -1.671109676361084,
  -1.5714483261108398,
  -1.5115957260131836,
  -1.500952959060669,
  -1.672407865524292,
  -1.6420881748199463,
  -1.1071901321411133,
  -1.372312068939209,
  -1.4877464771270752,
  -1.352004051208496,
  -1.625188946723

In [21]:
greedy_bleurt_mean = np.mean(greedy_bleurt_results['scores'])
sampling_bleurt_mean = np.mean(sampling_bleurt_results['scores'])

print("Greedy bleurt mean", greedy_bleurt_mean)
print("Sampling bleurt mean", sampling_bleurt_mean)

Greedy bleurt mean -1.2334054590231942
Sampling bleurt mean -1.2527204501214704


In [24]:
print("Min greedy value: ", np.min(greedy_bleurt_results.get('scores')))
print("Max greedy value: ", np.max(greedy_bleurt_results.get('scores')))

Min greedy value:  -2.026167631149292
Max greedy value:  -0.02947629988193512


In [25]:
print("Min sampling value: ", np.min(sampling_bleurt_results.get('scores')))
print("Max sampling value: ", np.max(sampling_bleurt_results.get('scores')))

Min sampling value:  -1.8047326803207397
Max sampling value:  -0.18105460703372955


In [ ]:
# BERTscore

greedy_bertscore_results = bertscore_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target'],
    lang="en"
)

sampling_bertscore_results = bertscore_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target'],
    lang="en"
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
greedy_bertscore_results

In [37]:
greedy_bertscore_f1_mean = np.mean(greedy_bertscore_results['f1'])
greedy_bertscore_precision_mean = np.mean(greedy_bertscore_results['precision'])
greedy_bertscore_recall_mean = np.mean(greedy_bertscore_results['recall'])

sampling_bertscore_f1_mean = np.mean(sampling_bertscore_results['f1'])
sampling_bertscore_precision_mean = np.mean(sampling_bertscore_results['precision'])
sampling_bertscore_recall_mean = np.mean(sampling_bertscore_results['recall'])

print("Greedy F1 mean", round(greedy_bertscore_f1_mean, 4))
print("Greedy precision mean", round(greedy_bertscore_precision_mean, 4))
print("Greedy recall mean", round(greedy_bertscore_recall_mean, 4))

print("Sampling F1 mean", round(sampling_bertscore_f1_mean, 4))
print("Sampling precision mean", round(sampling_bertscore_precision_mean, 4))
print("Sampling recall mean", round(sampling_bertscore_recall_mean, 4))


Greedy F1 mean 0.8422
Greedy precision mean 0.8222
Greedy recall mean 0.8638
Sampling F1 mean 0.8386
Sampling precision mean 0.8138
Sampling recall mean 0.8655


In [38]:
import os
os.chdir('nubia')

In [39]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.7MB 6.8MB/s 
     |████████████████████████████████| 184kB 20.0MB/s 
     |████████████████████████████████| 1.6MB 27.4MB/s 
     |████████████████████████████████| 153kB 41.1MB/s 
     |████████████████████████████████| 133kB 44.8MB/s 
     |████████████████████████████████| 215kB 39.7MB/s 
     |████████████████████████████████| 1.9MB 42.4MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 112kB 42.6MB/s 
     |████████████████████████████████| 7.7MB 42.0MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 962kB 36.8MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 3.2MB 42.0MB/s 
     |████████████████████████████████| 645kB 39.6MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=13c2d7332bb973a4fd8ed852c4d41c6f584884e635d66dbdee94c3ec68cf6d0e
  Stored in director

In [40]:
from nubia_score import Nubia
n = Nubia()

1042301B [00:00, 5379111.03B/s]
456318B [00:00, 4252804.55B/s]
100%|██████████| 548118077/548118077 [00:15<00:00, 35343696.81B/s]


In [41]:
greedy_nubia_contradiction = []
greedy_nubia_grammar_hyp = []
greedy_nubia_grammar_ref = []
greedy_nubia_logical_agreement = []
greedy_nubia_semantic_relation = []
greedy_nubia_score = []

for i, sentence in enumerate(zip(validation_data, greedy_generated_data)):
  greedy_nubia_result = n.score(
      greedy_generated_data[i],
      validation_data['target'][i],
      verbose=True, 
      get_features=True
  )

  greedy_nubia_contradiction.append(greedy_nubia_result.get('features').get('contradiction'))
  greedy_nubia_grammar_hyp.append(greedy_nubia_result.get('features').get('grammar_hyp'))
  greedy_nubia_grammar_ref.append(greedy_nubia_result.get('features').get('grammar_ref'))
  greedy_nubia_logical_agreement.append(greedy_nubia_result.get('features').get('logical_agreement'))
  greedy_nubia_semantic_relation.append(greedy_nubia_result.get('features').get('semantic_relation'))

  greedy_nubia_score.append(greedy_nubia_result.get('nubia_score'))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

NUBIA score: 0.9063733713847101/1.0
Semantic relation: 0.4754882752895355/5.0
Percent chance of contradiction: 4.295014962553978%
Percent chance of irrelevancy or new information: 92.13115572929382%
Percent chance of logical agreement: 3.573828935623169%

NUBIA score: 0.06420825369545502/1.0
Semantic relation: 0.7011223435401917/5.0
Percent chance of contradiction: 71.6114342212677%
Percent chance of irrelevancy or new information: 28.072476387023926%
Percent chance of logical agreement: 0.31608573626726866%

NUBIA score: 0.12689206942706577/1.0
Semantic relation: 0.4167957603931427/5.0
Percent chance of contradiction: 87.48197555541992%
Percent chance of irrelevancy or new information: 10.50075814127922%
Percent chance of logical agreement: 2.0172705873847008%

NUBIA score: 0.881669971055028/1.0
Semantic relation: 0.07223048806190491/5.0
Percent chance of contradiction: 70.64803838729858%
Percent chance of 

In [42]:
import numpy as np

greedy_nubia_contradiction
greedy_nubia_grammar_hyp
greedy_nubia_grammar_ref
greedy_nubia_logical_agreement
greedy_nubia_semantic_relation
greedy_nubia_score

greedy_nubia_contradiction_mean = np.mean(greedy_nubia_contradiction)
greedy_nubia_grammar_hyp_mean = np.mean(greedy_nubia_grammar_hyp)
greedy_nubia_grammar_ref_mean = np.mean(greedy_nubia_grammar_ref)
greedy_nubia_logical_agreement_mean = np.mean(greedy_nubia_logical_agreement)
greedy_nubia_semantic_relation_mean = np.mean(greedy_nubia_semantic_relation)
greedy_nubia_score_mean = np.mean(greedy_nubia_score)

print("Greedy nubia_contradiction mean", round(greedy_nubia_contradiction_mean, 4))
print("Greedy nubia_grammar_hyp mean", round(greedy_nubia_grammar_hyp_mean, 4))
print("Greedy nubia_grammar_ref mean", round(greedy_nubia_grammar_ref_mean, 4))
print("Greedy nubia_logical_agreement mean", round(greedy_nubia_logical_agreement_mean, 4))
print("Greedy nubia_semantic_relation mean", round(greedy_nubia_semantic_relation_mean, 4))
print("Greedy nubia_score mean", round(greedy_nubia_score_mean, 4))

Greedy nubia_contradiction mean 35.9945
Greedy nubia_grammar_hyp mean 4.5859
Greedy nubia_grammar_ref mean 6.1442
Greedy nubia_logical_agreement mean 8.0839
Greedy nubia_semantic_relation mean 0.8038
Greedy nubia_score mean 0.3959


In [43]:
sampling_nubia_contradiction = []
sampling_nubia_grammar_hyp = []
sampling_nubia_grammar_ref = []
sampling_nubia_logical_agreement = []
sampling_nubia_semantic_relation = []
sampling_nubia_score = []

for i, sentence in enumerate(zip(validation_data, sampling_generated_data)):
  sampling_nubia_result = n.score(
      sampling_generated_data[i],
      validation_data['target'][i],
      verbose=True, 
      get_features=True
  )

  sampling_nubia_contradiction.append(sampling_nubia_result.get('features').get('contradiction'))
  sampling_nubia_grammar_hyp.append(sampling_nubia_result.get('features').get('grammar_hyp'))
  sampling_nubia_grammar_ref.append(sampling_nubia_result.get('features').get('grammar_ref'))
  sampling_nubia_logical_agreement.append(sampling_nubia_result.get('features').get('logical_agreement'))
  sampling_nubia_semantic_relation.append(sampling_nubia_result.get('features').get('semantic_relation'))

  sampling_nubia_score.append(sampling_nubia_result.get('nubia_score'))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

NUBIA score: 0.9114304898153622/1.0
Semantic relation: 0.6791858673095703/5.0
Percent chance of contradiction: 74.90115761756897%
Percent chance of irrelevancy or new information: 22.047950327396393%
Percent chance of logical agreement: 3.050892613828182%

NUBIA score: 0.9054729669067514/1.0
Semantic relation: 0.6520488262176514/5.0
Percent chance of contradiction: 75.8093774318695%
Percent chance of irrelevancy or new information: 23.849110305309296%
Percent chance of logical agreement: 0.34151782747358084%

NUBIA score: 0.48656273201649514/1.0
Semantic relation: 0.8418130874633789/5.0
Percent chance of contradiction: 41.476044058799744%
Percent chance of irrelevancy or new information: 52.895718812942505%
Percent chance of logical agreement: 5.628242343664169%

NUBIA score: 0.3367404658338746/1.0
Semantic relation: -0.08461229503154755/5.0
Percent chance of contradiction: 97.93156385421753%
Percent chance 

In [44]:
sampling_nubia_contradiction
sampling_nubia_grammar_hyp
sampling_nubia_grammar_ref
sampling_nubia_logical_agreement
sampling_nubia_semantic_relation
sampling_nubia_score

sampling_nubia_contradiction_mean = np.mean(sampling_nubia_contradiction)
sampling_nubia_grammar_hyp_mean = np.mean(sampling_nubia_grammar_hyp)
sampling_nubia_grammar_ref_mean = np.mean(sampling_nubia_grammar_ref)
sampling_nubia_logical_agreement_mean = np.mean(sampling_nubia_logical_agreement)
sampling_nubia_semantic_relation_mean = np.mean(sampling_nubia_semantic_relation)
sampling_nubia_score_mean = np.mean(sampling_nubia_score)

print("Sampling nubia_contradiction mean", round(sampling_nubia_contradiction_mean, 4))
print("Sampling nubia_grammar_hyp mean", round(sampling_nubia_grammar_hyp_mean, 4))
print("Sampling nubia_grammar_ref mean", round(sampling_nubia_grammar_ref_mean, 4))
print("Sampling nubia_logical_agreement mean", round(sampling_nubia_logical_agreement_mean, 4))
print("Sampling nubia_semantic_relation mean", round(sampling_nubia_semantic_relation_mean, 4))
print("Sampling nubia_score mean", round(sampling_nubia_score_mean, 4))

Sampling nubia_contradiction mean 35.9636
Sampling nubia_grammar_hyp mean 4.5859
Sampling nubia_grammar_ref mean 6.305
Sampling nubia_logical_agreement mean 10.2197
Sampling nubia_semantic_relation mean 0.7439
Sampling nubia_score mean 0.5239


## Diversity (MSTTR, Distinct 1/2/3, Unique 1/2/3, Entropy 1/2/3)


In [45]:
# Mean segmental Type-Token Ratio (MSTTR)
from lexical_diversity import lex_div as ld

greedy_msttr_results = []
for i, sentence in enumerate(greedy_generated_data):
  greedy_flt = ld.flemmatize(greedy_generated_data[i])
  greedy_msttr_result = ld.msttr(greedy_flt)
  greedy_msttr_results.append(greedy_msttr_result)

sampling_msttr_results = []
for i, sentence in enumerate(sampling_generated_data):
  sampling_flt = ld.flemmatize(sampling_generated_data[i])
  sampling_msttr_result = ld.msttr(sampling_flt)
  sampling_msttr_results.append(sampling_msttr_result)


greedy_msttr_mean = np.mean(greedy_msttr_results)
sampling_msttr_mean = np.mean(sampling_msttr_results)

print("Greedy MSTTR mean", greedy_msttr_mean)
print("Sampling MSTTR ", sampling_msttr_mean)

Greedy MSTTR mean 0.8584854331165704
Sampling MSTTR  0.8808420442035308


In [46]:
# Distinct-n
from distinct_n import distinct_n_corpus_level

greedy_distinct_1_results = distinct_n_corpus_level(greedy_generated_data, 1)
sampling_distinct_1_results = distinct_n_corpus_level(sampling_generated_data, 1)

greedy_distinct_2_results = distinct_n_corpus_level(greedy_generated_data, 2)
sampling_distinct_2_results = distinct_n_corpus_level(sampling_generated_data, 2)

greedy_distinct_3_results = distinct_n_corpus_level(greedy_generated_data, 3)
#sampling_distinct_3_results = distinct_n_corpus_level(sampling_generated_data, 3)

print("Greedy distict 1-gram", greedy_distinct_1_results)
print("Greedy distict 2-gram", greedy_distinct_2_results)
print("Greedy distict 3-gram", greedy_distinct_3_results)

print("Sampling distict 1-gram", sampling_distinct_1_results)
print("Sampling distict 2-gram", sampling_distinct_2_results)
#print("Sampling distict 3-gram", sampling_distinct_3_results)

Greedy distict 1-gram 0.19073949441327304
Greedy distict 2-gram 0.594407569368344
Greedy distict 3-gram 0.8049323555514248
Sampling distict 1-gram 0.14742448599526795
Sampling distict 2-gram 0.5483514458089859


In [47]:
# Unique-n

import nltk

greedy_document = []
for sentence in greedy_generated_data:
  sentence = sentence.split()
  for word in sentence:
    greedy_document.append(word)

sampling_document = []
for sentence in sampling_generated_data:
  sentence = sentence.split()
  for word in sentence:
    sampling_document.append(word)

greedy_unigram_finder = nltk.FreqDist(greedy_document)
greedy_bigram_finder = nltk.FreqDist(nltk.bigrams(greedy_document))
greedy_trigram_finder = nltk.FreqDist(nltk.trigrams(greedy_document))

sampling_unigram_finder = nltk.FreqDist(sampling_document)
sampling_bigram_finder = nltk.FreqDist(nltk.bigrams(sampling_document))
sampling_trigram_finder = nltk.FreqDist(nltk.trigrams(sampling_document))

def unique_finder(finder):
  counter = 0
  for occurence in range(len(finder)):
    item = finder.popitem()
    if item[1] == 1:
      counter += 1
  return counter

greedy_uni_count = unique_finder(greedy_unigram_finder)
greedy_bi_count = unique_finder(greedy_bigram_finder)
greedy_tri_count = unique_finder(greedy_trigram_finder)

sampling_uni_count = unique_finder(sampling_unigram_finder)
sampling_bi_count = unique_finder(sampling_bigram_finder)
sampling_tri_count = unique_finder(sampling_trigram_finder)

print("Greedy unique unigrams", greedy_uni_count)
print("Greedy unique bigrams", greedy_bi_count)
print("Greedy unique trigrams", greedy_tri_count)

print("Sampling unique unigrams", sampling_uni_count)
print("Sampling unique bigrams", sampling_bi_count)
print("Sampling unique trigrams", sampling_tri_count)

Greedy unique unigrams 972
Greedy unique bigrams 13115
Greedy unique trigrams 28157
Sampling unique unigrams 1285
Sampling unique bigrams 20540
Sampling unique trigrams 40161


In [48]:
# Entropy-n
from scipy import stats

greedy_unigram_finder = nltk.FreqDist(greedy_document)
greedy_bigram_finder = nltk.FreqDist(nltk.bigrams(greedy_document))
greedy_trigram_finder = nltk.FreqDist(nltk.trigrams(greedy_document))

sampling_unigram_finder = nltk.FreqDist(sampling_document)
sampling_bigram_finder = nltk.FreqDist(nltk.bigrams(sampling_document))
sampling_trigram_finder = nltk.FreqDist(nltk.trigrams(sampling_document))

greedy_unigram_probabilities = [v for k,v in greedy_unigram_finder.items()]
greedy_bigram_probabilities = [v for k,v in greedy_bigram_finder.items()]
greedy_trigram_probabilities = [v for k,v in greedy_trigram_finder.items()]

sampling_unigram_probabilities = [v for k,v in sampling_unigram_finder.items()]
sampling_bigram_probabilities = [v for k,v in sampling_bigram_finder.items()]
sampling_trigram_probabilities = [v for k,v in sampling_trigram_finder.items()]

greedy_entropy_1 = stats.entropy(greedy_unigram_probabilities)
greedy_entropy_2 = stats.entropy(greedy_bigram_probabilities)
greedy_entropy_2 = stats.entropy(greedy_trigram_probabilities)

sampling_entropy_1 = stats.entropy(sampling_unigram_probabilities)
sampling_entropy_2 = stats.entropy(sampling_bigram_probabilities)
sampling_entropy_2 = stats.entropy(sampling_trigram_probabilities)

print("Greedy entropy-1:", greedy_entropy_1)
print("Greedy entropy-2:", greedy_entropy_2)
#print("Greedy entropy-3:", greedy_entropy_3)

print("Sampling entropy-1:", sampling_entropy_1)
print("Sampling entropy-2:", sampling_entropy_2)
#print("Sampling entropy-3:", sampling_entropy_3)

Greedy entropy-1: 5.818471485143353
Greedy entropy-2: 10.24192371889163
Sampling entropy-1: 6.123031317238445
Sampling entropy-2: 10.602803025214007
